<a href="https://colab.research.google.com/github/Lucia2323/Time-coefficient-value-for-attention-head/blob/main/RSA_mapping_model_load_hp_fmri_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WHEN CLOSURE, SAVE IT ON GITHUB

To run this colab, you first need to upload the harry potter dataset into your google drive and give colab access to my Drive.

Note: In placed a link to the HP_data folder in my own drive, and replaced all occurrences of 'HPfmri/' in the code with 'HP_data/fMRI/', I got a warning about being connected to a GPU runtime, and wasting it. I don't know how important that is - perhaps better to save GPU time for when we need it.

In [ ]:
from google.colab import drive
from google.colab import files
import numpy as np
from scipy.stats import zscore


In [ ]:
#@title Mount google drive (where the data is uploaded).

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
subject_id = 'K' #@param: so the subjects are parameters
subject_data = np.load(f'/content/drive/My Drive/HP_data/fMRI/data_subject_{subject_id}.npy')
time_fmri = np.load('/content/drive/My Drive/HP_data/fMRI/time_fmri.npy')
runs_fmri = np.load('/content/drive/My Drive/HP_data/fMRI/runs_fmri.npy') #a trial? 
words_fmri = np.load('/content/drive/My Drive/HP_data/fMRI/words_fmri.npy')
time_words_fmri = np.load('/content/drive/My Drive/HP_data/fMRI/time_words_fmri.npy')

In [ ]:
#delay = time in seconds between the first of the four words is shown to the subject and the fMRI scan is taken ->
#match each four-words to its scan by a timelime: to estimate in seconds the time between each four words-scan 
#(standard brain data preprocessing procedure)
def delay_one(mat, d): 
  """delays a matrix by a delay d. Positive d ==> row t (row at the time the scan was taken) has row t-d."""
  new_mat = np.zeros_like(mat) #Return an array of zeros with the same shape and type as a given array.
  if d>0:
      new_mat[d:] = mat[:-d]
  elif d<0:
      new_mat[:d] = mat[-d:]
  else:
      new_mat = mat
  return new_mat


def delay_mat(mat, delays):
  """delays a matrix by a set of delays d.
    a row t in the returned matrix has the concatenated:
    row(t-delays[0],t-delays[1]...t-delays[last] ).
  """
  new_mat = np.concatenate([delay_one(mat, d) for d in delays],axis = -1)
  return new_mat


#we look at similarities between the model and brain representations for the words until EOS (end of a sentence) 
#token is reached       
def prepare_nlp_features(train_features, test_features, word_train_indicator, TR_train_indicator, SKIP_WORDS=20, END_WORDS=5176):
    time = np.load('/content/drive/My Drive/HP_data/fMRI/time_fmri.npy')
    runs = np.load('/content/drive/My Drive/HP_data/fMRI/runs_fmri.npy') 
    time_words = np.load('/content/drive/My Drive/HP_data/fMRI/time_words_fmri.npy')
    time_words = time_words[SKIP_WORDS:END_WORDS]
    #'SKIP_WORDS'?
        
    words_id = np.zeros([len(time_words)])
    # w=find what TR (trial?) each word belongs to
    for i in range(len(time_words)):
        words_id[i] = np.where(time_words[i]> time)[0][-1]
        
    all_features = np.zeros([time_words.shape[0], train_features.shape[1]])
    all_features[word_train_indicator] = train_features
    all_features[~word_train_indicator] = test_features
        
    p = all_features.shape[1]
    tmp = np.zeros([time.shape[0], p])
    for i in range(time.shape[0]):
        tmp[i] = np.mean(all_features[(words_id<=i)*(words_id>i-1)],0)
    tmp = delay_mat(tmp, np.arange(1,5))

    # remove the edges of each run(?)
    tmp = np.vstack([zscore(tmp[runs==i][20:-15]) for i in range(1,5)])
    tmp = np.nan_to_num(tmp)
        
    return tmp[TR_train_indicator], tmp[~TR_train_indicator]


#K-fold cross-validation? 
def get_fold_flags(n, n_folds):
    flags = np.zeros((n))
    num_items_in_each_fold = int(np.floor(n/n_folds))
    for i in range(0,n_folds -1):
        flags[i*num_items_in_each_fold:(i+1)*num_items_in_each_fold] = i
    flags[(n_folds-1)*num_items_in_each_fold:] = (n_folds-1)
    return flags

def tr_to_word_train_indicator(tr_train_indicator, skip_words=20, end_words=5176):
    time = np.load('/content/drive/My Drive/HP_data/fMRI/time_fmri.npy')
    runs = np.load('/content/drive/My Drive/HP_data/fMRI/runs_fmri.npy') 
    time_words = np.load('/content/drive/My Drive/HP_data/fMRI/time_words_fmri.npy')
    time_words = time_words[skip_words:end_words]
        
    word_train_indicator = np.zeros([len(time_words)], dtype=bool)    
    words_id = np.zeros([len(time_words)],dtype=int)
    # Find what TR each word belongs to.
    for i in range(len(time_words)):                
        words_id[i] = np.where(time_words[i]> time)[0][-1]
        
        if words_id[i] <= len(runs) - 15:
            offset = runs[int(words_id[i])]*20 + (runs[int(words_id[i])]-1)*15
            if tr_train_indicator[int(words_id[i])-offset-1] == 1:
                word_train_indicator[i] = True
    return word_train_indicator 

In [ ]:
#a brain scan every four words
#time notation: datapoints of the Harry Potter and brain scan datasets respect chronological order
# - each word from the Harry Potter dataset has a number which corresponds to its chronological order within the text/dataset
# - each fMRI scan from the brain dataset has a number which corresponds to its chronological order among the recordings
# --> every four words (their numbers) are matched with the number of the corresponding brain scan
n_words = subject_data.shape[0]
n_voxels = subject_data.shape[1]
print('Number of voxels for this subject is {n_voxels} for {n_words} scans.')

Number of voxels for this subject is {n_voxels} for {n_words} scans.


In [ ]:
n_folds = 4
test_fold = 1
skip = 5
fold_flags = get_fold_flags(n_words, n_folds=n_folds)
train_ind = fold_flags!=test_fold
test_ind = fold_flags==test_fold
train_indicator = tr_to_word_train_indicator(train_ind)

In [ ]:
#@title Get NLP features (currently it is just a random matrix(?))
SKIP_WORDS = 20
END_WORDS = 5176

# You need to replace this with the embeddings that you get from any NLP model for words_fmri.
nlp_features = np.random.uniform(size=(len(words_fmri), 100))


train_nlp_features = nlp_features[SKIP_WORDS:END_WORDS,:][train_indicator]
test_nlp_features = nlp_features[SKIP_WORDS:END_WORDS,:][~train_indicator]
train_features, test_features = prepare_nlp_features(train_nlp_features, test_nlp_features, train_indicator, train_ind)

print(train_features.shape, test_features.shape)

(909, 400) (302, 400)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [ ]:
#@title Get fmri data points.

train_data = subject_data[train_ind]
test_data = subject_data[test_ind]

In [ ]:
#@title Filter out some of data points in the margins of test and train sets.

# skip TRs between train and test data
if test_fold == 0: # just remove from front end
    train_data = train_data[skip:,:]
    train_features = train_features[skip:,:]
elif test_fold == n_folds-1: # just remove from back end
    train_data = train_data[:-skip,:]
    train_features = train_features[:-skip,:]
else:
    test_data = test_data[skip:-skip,:]
    test_features = test_features[skip:-skip,:]

# normalize data
train_data = np.nan_to_num(zscore(np.nan_to_num(train_data)))
test_data = np.nan_to_num(zscore(np.nan_to_num(test_data)))

In [ ]:
print('fmri features:', train_data.shape, test_data.shape)
print('nlp features:', train_features.shape, test_features.shape)
#why first dimensions are the same?

fmri features: (909, 25003) (292, 25003)
nlp features: (909, 400) (292, 400)


# Representational Similarity

In [ ]:
def dot_product_rsa(reps1, reps2):
  """Compute representational similarity between two sets of representations.

  Args:
    reps1: float array; [num_examples, feature_size1]
    reps2: float array; [num_examples, feature_size2]
  """
  assert reps1.shape[0] == reps2.shape[0], 'First dimensions of inputs should match.'

  # Normalize input representations
  reps1 = reps1 / np.linalg.norm(reps1)
  reps2 = reps2 / np.linalg.norm(reps2)

  # Compute and normalize similarity matrices:
  similarities1 = reps1.dot(reps1.T) #instead of dot product you can use any similarity measure. e.g. matrix product a*b=a*bT
  similarities2 = reps2.dot(reps2.T) #dot is relative so has to be standardized - what about Pearson r?
  similarities1 = similarities1 / np.linalg.norm(similarities1)
  similarities2 = similarities2 / np.linalg.norm(similarities2)

  # Here you can do pearson-r instead of this (or again any measure of similarity).
  similarity_of_similarity = np.sum(similarities1 * similarities2, axis=-1)
  return np.sum(similarity_of_similarity)

In [ ]:
def dot_product_rsa_for_lists_of_reps(reps):
  """Compute representational similarity between two sets of representations.

  Args:
    reps: list of float arrays; List of arrrays with shape [num_examples, feature_size]
      (feature size can be different for each item in the list).
  """
  norm_reps = []
  for rep in reps:
  # Normalize input representations
    norm_reps.append(rep / np.linalg.norm(rep))

  # Compute and normalize similarity matrices:
  similarity_matrices = []
  for rep in norm_reps:
    similarities = rep.dot(rep.T) # instead of dot product you can use any similarity measure
    similarities = similarities / np.linalg.norm(similarities)
    similarity_matrices.append(similarities)

  # Shape of similarity_matrices: [num_of_rep_spaces, num_examples, num_examples]
  similarity_matrices = np.asarray(similarity_matrices) 
  import jax.numpy as jnp
  sim_of_sim_mat = jnp.zeros((similarity_matrices.shape[0], 
                              similarity_matrices.shape[0]))
  for i in np.arange(sim_of_sim_mat.shape[0]):
    for j in np.arange(sim_of_sim_mat.shape[1]):
      # Here you can do pearson-r instead of this (or again any measure of similarity).
      similarity_of_similarity = np.sum(
          similarity_matrices[i] * similarity_matrices[j], axis=-1)
      similarity_score = np.sum(similarity_of_similarity)
      sim_of_sim_mat[i][j] = similarity_score
  return sim_of_sim_mat

In [ ]:
dot_product_rsa(train_data, train_data)

1.0000000000000009

In [ ]:
dot_product_rsa_for_lists_of_reps([train_data, train_data])

TypeError: ignored

In [ ]:
dot_product_rsa(train_data, train_features)

0.05214565708629563

In [ ]:
dot_product_rsa(train_features, train_features)

1.0000000000000009

In [ ]:
#@title Compare two subjects:

subject_id_1 = 'K' #@param 
subject_data_1 = np.load(f'/content/drive/My Drive/HP_data/fMRI/data_subject_{subject_id_1}.npy')

subject_id_2 = 'F' #@param 
subject_data_2 = np.load(f'/content/drive/My Drive/HP_data/fMRI/data_subject_{subject_id_2}.npy')

subject_data_1 = np.nan_to_num(zscore(np.nan_to_num(subject_data_1)))
subject_data_2 = np.nan_to_num(zscore(np.nan_to_num(subject_data_2)))

print(f'Representational similarity between subject {subject_id_1} and {subject_id_2} is:', dot_product_rsa(subject_data_1, subject_data_2))

Representational similarity between subject K and F is: 0.012922335085487204
